<a href="https://colab.research.google.com/github/kartikeyaGUPTA45/AUTOMATED-MACHINE-LEARNING/blob/master/FEATURETOOLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FEATURE TOOLS

# INSTALLATION

In [ ]:
! pip install featuretools

# IMPORTING FEATURETOOLS AND OTHER NECESSARY LIBRARIES 

In [ ]:
import featuretools as ft
import numpy as np
import pandas as pd

# IMPORTING THE DATASET

In [ ]:
train = pd.read_csv("/content/train_v9rqX0R.csv")
test = pd.read_csv("/content/test_AbJTz2l.csv")

In [ ]:
print(train)

     Item_Identifier  Item_Weight  ...        Outlet_Type  Item_Outlet_Sales
0              FDA15        9.300  ...  Supermarket Type1          3735.1380
1              DRC01        5.920  ...  Supermarket Type2           443.4228
2              FDN15       17.500  ...  Supermarket Type1          2097.2700
3              FDX07       19.200  ...      Grocery Store           732.3800
4              NCD19        8.930  ...  Supermarket Type1           994.7052
...              ...          ...  ...                ...                ...
8518           FDF22        6.865  ...  Supermarket Type1          2778.3834
8519           FDS36        8.380  ...  Supermarket Type1           549.2850
8520           NCJ29       10.600  ...  Supermarket Type1          1193.1136
8521           FDN46        7.210  ...  Supermarket Type2          1845.5976
8522           DRG01       14.800  ...  Supermarket Type1           765.6700

[8523 rows x 12 columns]


In [ ]:
test_Item_Identifier = test['Item_Identifier']
test_Outlet_Identifier = test['Outlet_Identifier']
sales = train['Item_Outlet_Sales']
train.drop(['Item_Outlet_Sales'], axis=1, inplace=True)

In [ ]:
print(train)

     Item_Identifier  Item_Weight  ... Outlet_Location_Type        Outlet_Type
0              FDA15        9.300  ...               Tier 1  Supermarket Type1
1              DRC01        5.920  ...               Tier 3  Supermarket Type2
2              FDN15       17.500  ...               Tier 1  Supermarket Type1
3              FDX07       19.200  ...               Tier 3      Grocery Store
4              NCD19        8.930  ...               Tier 3  Supermarket Type1
...              ...          ...  ...                  ...                ...
8518           FDF22        6.865  ...               Tier 3  Supermarket Type1
8519           FDS36        8.380  ...               Tier 2  Supermarket Type1
8520           NCJ29       10.600  ...               Tier 2  Supermarket Type1
8521           FDN46        7.210  ...               Tier 3  Supermarket Type2
8522           DRG01       14.800  ...               Tier 1  Supermarket Type1

[8523 rows x 11 columns]


In [ ]:
combine = train.append(test, ignore_index=True)

In [ ]:
print(combine)

      Item_Identifier  Item_Weight  ... Outlet_Location_Type        Outlet_Type
0               FDA15         9.30  ...               Tier 1  Supermarket Type1
1               DRC01         5.92  ...               Tier 3  Supermarket Type2
2               FDN15        17.50  ...               Tier 1  Supermarket Type1
3               FDX07        19.20  ...               Tier 3      Grocery Store
4               NCD19         8.93  ...               Tier 3  Supermarket Type1
...               ...          ...  ...                  ...                ...
14199           FDB58        10.50  ...               Tier 1  Supermarket Type1
14200           FDD47         7.60  ...               Tier 3  Supermarket Type2
14201           NCO17        10.00  ...               Tier 2  Supermarket Type1
14202           FDJ26        15.30  ...               Tier 2  Supermarket Type1
14203           FDU37         9.50  ...               Tier 2  Supermarket Type1

[14204 rows x 11 columns]


In [ ]:
combine.isnull().sum()

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [ ]:
print(combine.Outlet_Size)

0        Medium
1        Medium
2        Medium
3           NaN
4          High
          ...  
14199     Small
14200    Medium
14201       NaN
14202       NaN
14203       NaN
Name: Outlet_Size, Length: 14204, dtype: object


In [ ]:
combine['Item_Weight'].fillna(combine['Item_Weight'].mean(), inplace = True)
combine['Outlet_Size'].fillna("missing", inplace = True)

In [ ]:
combine.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

In [ ]:
combine['Item_Fat_Content'].value_counts()

Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

In [ ]:
fat_content_dict = {'Low Fat':0, 'Regular':1, 'LF':0, 'reg':1, 'low fat':0}

combine['Item_Fat_Content'] = combine['Item_Fat_Content'].replace(fat_content_dict, regex=True)

In [ ]:
combine['id'] = combine['Item_Identifier'] + combine['Outlet_Identifier']
combine.drop(['Item_Identifier'], axis=1, inplace=True)

In [ ]:
# creating and entity set 'es'
es = ft.EntitySet(id = 'sales')

# adding a dataframe 
es.entity_from_dataframe(entity_id = 'bigmart', dataframe = combine, index = 'id')

Entityset: sales
  Entities:
    bigmart [Rows: 14204, Columns: 11]
  Relationships:
    No relationships

In [ ]:
es.normalize_entity(base_entity_id='bigmart', new_entity_id='outlet', index = 'Outlet_Identifier', 
additional_variables = ['Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type'])

Entityset: sales
  Entities:
    bigmart [Rows: 14204, Columns: 7]
    outlet [Rows: 10, Columns: 5]
  Relationships:
    bigmart.Outlet_Identifier -> outlet.Outlet_Identifier

In [ ]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
target_entity = 'bigmart', 
max_depth = 2, 
verbose = 1)

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO - Scheduler closing...
distributed.scheduler - INFO - Scheduler closing all comms


Built 37 features
Elapsed: 00:02 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


In [ ]:
feature_matrix.columns

Index(['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type',
       'Item_MRP', 'Outlet_Identifier', 'outlet.Outlet_Establishment_Year',
       'outlet.Outlet_Size', 'outlet.Outlet_Location_Type',
       'outlet.Outlet_Type', 'outlet.SUM(bigmart.Item_Weight)',
       'outlet.SUM(bigmart.Item_Fat_Content)',
       'outlet.SUM(bigmart.Item_Visibility)', 'outlet.SUM(bigmart.Item_MRP)',
       'outlet.STD(bigmart.Item_Weight)',
       'outlet.STD(bigmart.Item_Fat_Content)',
       'outlet.STD(bigmart.Item_Visibility)', 'outlet.STD(bigmart.Item_MRP)',
       'outlet.MAX(bigmart.Item_Weight)',
       'outlet.MAX(bigmart.Item_Fat_Content)',
       'outlet.MAX(bigmart.Item_Visibility)', 'outlet.MAX(bigmart.Item_MRP)',
       'outlet.SKEW(bigmart.Item_Weight)',
       'outlet.SKEW(bigmart.Item_Fat_Content)',
       'outlet.SKEW(bigmart.Item_Visibility)', 'outlet.SKEW(bigmart.Item_MRP)',
       'outlet.MIN(bigmart.Item_Weight)',
       'outlet.MIN(bigmart.Item_Fat_Content)',
       

In [ ]:
feature_matrix.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,outlet.Outlet_Establishment_Year,outlet.Outlet_Size,outlet.Outlet_Location_Type,outlet.Outlet_Type,outlet.SUM(bigmart.Item_Weight),outlet.SUM(bigmart.Item_Fat_Content),outlet.SUM(bigmart.Item_Visibility),outlet.SUM(bigmart.Item_MRP),outlet.STD(bigmart.Item_Weight),outlet.STD(bigmart.Item_Fat_Content),outlet.STD(bigmart.Item_Visibility),outlet.STD(bigmart.Item_MRP),outlet.MAX(bigmart.Item_Weight),outlet.MAX(bigmart.Item_Fat_Content),outlet.MAX(bigmart.Item_Visibility),outlet.MAX(bigmart.Item_MRP),outlet.SKEW(bigmart.Item_Weight),outlet.SKEW(bigmart.Item_Fat_Content),outlet.SKEW(bigmart.Item_Visibility),outlet.SKEW(bigmart.Item_MRP),outlet.MIN(bigmart.Item_Weight),outlet.MIN(bigmart.Item_Fat_Content),outlet.MIN(bigmart.Item_Visibility),outlet.MIN(bigmart.Item_MRP),outlet.MEAN(bigmart.Item_Weight),outlet.MEAN(bigmart.Item_Fat_Content),outlet.MEAN(bigmart.Item_Visibility),outlet.MEAN(bigmart.Item_MRP),outlet.COUNT(bigmart),outlet.NUM_UNIQUE(bigmart.Item_Type),outlet.MODE(bigmart.Item_Type)
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DRA12OUT010,11.600000,0,0.068535,Soft Drinks,143.0154,OUT010,1998,missing,Tier 3,Grocery Store,11768.655000,330,94.293418,130572.7618,4.675070,0.479301,0.073604,62.010835,21.350000,1,0.313935,266.6884,0.112759,0.599012,0.776902,0.104693,4.610000,0,0.0,32.6558,12.722870,0.356757,0.101939,141.159742,925,16,Fruits and Vegetables
DRA12OUT013,11.600000,0,0.040912,Soft Drinks,142.3154,OUT013,1987,High,Tier 3,Supermarket Type1,19859.980000,549,93.555174,219172.4492,4.650214,0.478213,0.044005,62.140848,21.350000,1,0.185913,266.6884,0.104392,0.613449,0.759033,0.130888,4.555000,0,0.0,31.4900,12.788139,0.353509,0.060242,141.128428,1553,16,Fruits and Vegetables
DRA12OUT017,11.600000,0,0.041178,Soft Drinks,140.3154,OUT017,2007,missing,Tier 2,Supermarket Type1,19722.750000,544,94.342210,217561.3500,4.655234,0.477922,0.044152,62.295513,21.350000,1,0.188620,266.8884,0.106563,0.617805,0.774783,0.129950,4.555000,0,0.0,32.0900,12.782080,0.352560,0.061142,140.998931,1543,16,Snack Foods
DRA12OUT018,11.600000,0,0.041113,Soft Drinks,142.0154,OUT018,2009,Medium,Tier 3,Supermarket Type2,19794.425000,547,92.723425,217987.3906,4.650874,0.478308,0.044489,62.022851,21.350000,1,0.188323,266.3226,0.102602,0.612046,0.783017,0.133528,4.555000,0,0.0,31.8900,12.803638,0.353816,0.059976,141.000899,1546,16,Fruits and Vegetables
DRA12OUT027,12.792854,0,0.040748,Soft Drinks,140.0154,OUT027,1985,Medium,Tier 3,Supermarket Type3,19944.059742,551,94.075671,219838.2488,0.000000,0.478189,0.044228,62.059660,12.792854,1,0.186650,265.2884,0.000000,0.613802,0.774028,0.129638,12.792854,0,0.0,31.2900,12.792854,0.353432,0.060344,141.012347,1559,16,Fruits and Vegetables


In [ ]:
feature_matrix = feature_matrix.reindex(index=combine['id'])
feature_matrix = feature_matrix.reset_index()